<a href="https://colab.research.google.com/github/PedroAlvesDev/Search-Assistant/blob/main/C%C3%B3pia_de_Imers%C3%A3o_IA_Alura_%2B_Google_Gemini_Aula_05_Agentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install google-genai

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A Alura ainda não divulgou as datas das próximas edições da Imersão IA com Google Gemini. A Imersão mais recente ocorreu em janeiro de 2024.

A melhor forma de ficar por dentro das novidades e futuras edições é:

*   **Acompanhar as redes sociais da Alura:** Fique de olho nas páginas da Alura no Instagram, LinkedIn, Twitter e Facebook.
*   **Assinar a newsletter da Alura:** Cadastre seu e-mail no site da Alura para receber informações sobre os próximos eventos e cursos.
*   **Visitar o site da Alura:** Consulte regularmente o site da Alura para verificar se há alguma novidade sobre a Imersão IA com Google Gemini.

In [ ]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    config={"tools":[{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A próxima Imersão IA com Google Gemini da Alura aconteceu entre os dias 12 e 16 de maio de 2025. As inscrições para esta edição já encerraram no dia 11 de maio de 2025.

In [ ]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['próxima Imersão IA com Google Gemini Alura']
Páginas utilizadas na resposta: starten.tech, alura.com.br



In [ ]:
# Instalar Framework ADK de agentes do Google ################################################
%pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################

def agente_buscador(topico, data_de_hoje):
  buscador = Agent(
      name='agente_buscador',
      model='gemini-2.0-flash',
      instruction="""
      Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do google (google_search) para recuperar as últimas noticias de lançamentos muito relevantes sobre o tópico abaixo.
      Foque em no máximo 5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias sobre ele.
      Se um tema tiver poucas notícias ou reações entusiasmadas, é possível que ele não seja tão relevante assim e pode ser substituído por outro que tenha mais.
      Esses lançamentos relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
      """,
      description='Agente que busca informações no Google',
      tools=[google_search],
  )

  entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
  lancamentos = call_agent(buscador, entrada_do_agente_buscador)
  return lancamentos

In [ ]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um planejador de conteúdo, especialista em redes sociais. Com base na lista de lançamentos mais recentes e relevantes buscador, você deve:
        Usar a ferramenta de busca do Google (google_search) para criar um plano sobre quais são os pontos mais relevantes que poderíamos abordar em um post sobre cada um deles. Você também pode usar o (google_search) para encontrar mais informações sobre os temas e aprofundar.
        Ao final, você irá escolher o tema mais relevante entre eles com base nas suas pesquisas e retornar esse tema, seus pontos mais relevantes, e um plano com os assuntos a serem abordados no post que será escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [ ]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [ ]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [ ]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################

if not topico:
  print("Você esqueceu de digitar o tópico!")
else:
    print(f"Maravilha! Vamos então criar o post sobre novidades em {topico}")

    lancamentos_buscados = agente_buscador(topico, data_de_hoje)
    print("\n--- Resultado do Agente 1 (Buscador) ---\n")
    display(to_markdown(lancamentos_buscados))
    print("---------------------------------------------------------------")

    plano_de_post = agente_planejador(topico, lancamentos_buscados)
    print("\n--- Resultado do Agente 2 (Planejador) ---\n")
    display(to_markdown(plano_de_post))
    print("---------------------------------------------------------------")

    rascunho_de_post = agente_redator(topico, plano_de_post)
    print("\n--- Resultado do Agente 3 (Redator) ---\n")
    display(to_markdown(rascunho_de_post))
    print("---------------------------------------------------------------")

    post_final = agente_revisor(topico, rascunho_de_post)
    print("\n--- Resultado do Agente 4 (Revisor) ---\n")
    display(to_markdown(post_final))
    print("---------------------------------------------------------------")

🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: Agentes de IA
Maravilha! Vamos então criar o post sobre novidades em Agentes de IA

--- Resultado do Agente 1 (Buscador) ---



> Para encontrar os lançamentos mais relevantes e recentes sobre agentes de IA, vou realizar algumas buscas no Google. O foco será em novidades de abril e maio de 2025 que tenham gerado grande interesse e cobertura na mídia.
> 
> 
> Com base nas minhas buscas, aqui estão 5 lançamentos e desenvolvimentos recentes e relevantes sobre agentes de IA, considerando o período de aproximadamente um mês anterior a 19 de maio de 2025:
> 
> 1.  **Microsoft Build 2025 e a Era dos Agentes de IA**: A Microsoft anunciou inovações que marcam o início da era dos agentes de IA e da "open agentic web" durante o Microsoft Build 2025. Empresas como Fujitsu e NTT DATA estão utilizando o Azure AI Foundry para construir e gerenciar aplicações e agentes de IA. (Fonte: [12])
> 2.  **IBM watsonx Orchestrate**: A IBM anunciou atualizações em sua plataforma watsonx Orchestrate, oferecendo soluções para integrar, customizar e implantar agentes de IA. A plataforma inclui agentes pré-construídos para RH, vendas e compras, com integração a diversas aplicações empresariais. (Fonte: [4])
> 3.  **OpenAI lança guia para construção de agentes de IA**: A OpenAI publicou um guia prático sobre como construir agentes de IA autônomos, definindo-os como sistemas que utilizam modelos de linguagem (LLMs) para tomar decisões e executar tarefas de forma independente, acessando ferramentas externas como navegadores ou APIs. (Fonte: [5])
> 4.  **Adoção de agentes de IA no Brasil**: A B3 anunciou que vai massificar o uso de agentes de IA em seus produtos e serviços. (Fonte: [13]) Além disso, o Brasil acompanha essa tendência com o Plano Brasileiro de Inteligência Artificial. Empresas estão investindo fortemente no aprimoramento desses sistemas, tornando-os mais personalizados e eficientes. No Brasil, o investimento em IA por bancos e empresas estatais já ultrapassou R$ 2 bilhões, com a Financiadora de Estudos e Projetos (Finep) liderando os repasses e destinando R$ 1,4 bilhão para projetos de IA entre 2023 e junho de 2024. (Fonte: [21])
> 5.  **Tendências e aplicações de agentes de IA**: Agentes de IA estão sendo utilizados para automação de negócios, melhoria da eficiência operacional e tomada de decisões em tempo real. Eles são capazes de lidar com tarefas complexas de forma autônoma e interagir com sistemas e pessoas. (Fontes: [1], [4], [9])
> 
> 


---------------------------------------------------------------

--- Resultado do Agente 2 (Planejador) ---



> Okay, com base nos lançamentos sobre Agentes de IA que você forneceu, vou criar um plano para um post de mídia social abordando os temas mais relevantes.
> 
> **Ferramentas de busca:**
> 
> Com base nas informações coletadas, o tema mais relevante para um post sobre agentes de IA é:
> 
> **Tema:** Adoção e Tendências de Agentes de IA em 2025: O Que Você Precisa Saber
> 
> **Pontos Relevantes:**
> 
> *   **Crescimento e Adoção:**
>     *   A Microsoft Build 2025 marcou o início da era dos agentes de IA.
>     *   Empresas estão utilizando plataformas como Azure AI Foundry e watsonx Orchestrate para construir e gerenciar agentes de IA.
>     *   A B3 (Bolsa de Valores Brasileira) vai massificar o uso de agentes de IA em seus produtos e serviços.
>     *   O Brasil acompanha essa tendência com investimentos significativos em projetos de IA.
> *   **Funcionalidades e Aplicações:**
>     *   Agentes de IA podem tomar decisões e executar tarefas de forma autônoma, acessando ferramentas externas.
>     *   Aplicações incluem automação de negócios, melhoria da eficiência operacional e tomada de decisões em tempo real.
>     *   A Microsoft está desenvolvendo agentes de IA que podem operar em contextos individuais, organizacionais e de equipe.
>     *   Novos recursos no Microsoft Edge e Outlook, como tradução de PDFs e auxílio na leitura de e-mails, são impulsionados por agentes de IA.
> *   **Construção e Desenvolvimento:**
>     *   A OpenAI lançou um guia prático sobre como construir agentes de IA autônomos.
>     *   IBM watsonx Orchestrate oferece soluções para integrar, customizar e implantar agentes de IA sem codificação.
>     *   Componentes essenciais incluem modelos de linguagem (LLMs), ferramentas e instruções.
> *   **Impacto e Futuro:**
>     *   Agentes de IA estão transformando a forma como as empresas operam, liberando funcionários para trabalhos intelectuais.
>     *   A adoção de IA pode adicionar um impacto considerável ao PIB do Brasil.
>     *   A tendência é que os agentes de IA se tornem cada vez mais autônomos, colaborativos e personalizados.
> 
> **Plano para o Post:**
> 
> 1.  **Introdução:**
>     *   Definir o que são agentes de IA e sua importância crescente.
>     *   Mencionar a Microsoft Build 2025 como um marco para a era dos agentes de IA.
> 2.  **Adoção e Tendências no Brasil:**
>     *   Discutir como a B3 está adotando agentes de IA e os investimentos em IA no Brasil.
>     *   Referenciar o Plano Brasileiro de Inteligência Artificial.
> 3.  **Funcionalidades e Aplicações Práticas:**
>     *   Exemplos de como agentes de IA estão sendo usados em diferentes setores (finanças, RH, vendas).
>     *   Destacar os novos recursos no Microsoft Edge e Outlook.
> 4.  **Plataformas e Ferramentas para Construir Agentes de IA:**
>     *   Apresentar o guia da OpenAI para construção de agentes de IA.
>     *   Mencionar a plataforma IBM watsonx Orchestrate e suas capacidades "no-code".
>     *   Discutir os componentes essenciais para construir um agente de IA eficaz (LLMs, ferramentas, instruções).
> 5.  **Impacto e Considerações Futuras:**
>     *   Analisar o impacto dos agentes de IA na eficiência operacional e na transformação do trabalho.
>     *   Discutir o potencial impacto no PIB do Brasil.
>     *   Abordar a importância da ética, segurança e governança no uso de agentes de IA.
> 6.  **Conclusão:**
>     *   Reafirmar a importância dos agentes de IA como uma tendência tecnológica crucial.
>     *   Incentivar os leitores a explorar e implementar agentes de IA em suas próprias organizações.
>     *   Finalizar com uma pergunta para engajar a audiência (ex: "Como você vê o futuro dos agentes de IA na sua área de atuação?").
> 
> Este plano busca fornecer um panorama completo e informativo sobre os agentes de IA, destacando sua relevância, aplicações práticas e o que esperar para o futuro.
> 


---------------------------------------------------------------

--- Resultado do Agente 3 (Redator) ---



> 🚀 A Era dos Agentes de IA Chegou! 🤖✨
> 
> Já ouviu falar dos Agentes de Inteligência Artificial? 🤔 Na Microsoft Build 2025, ficou claro: eles são o futuro! Empresas como a B3 estão massificando o uso, e o Brasil não fica para trás com altos investimentos em IA. 🇧🇷
> 
> ⚙️ O que eles fazem? Agentes de IA tomam decisões e automatizam tarefas, turbinando a eficiência e a tomada de decisões em tempo real. Imagine no RH, finanças, vendas... As possibilidades são infinitas! 🤩
> 
> 🛠️ Quer construir o seu? A OpenAI lançou um guia prático e a IBM watsonx Orchestrate oferece soluções "no-code". Componentes-chave: LLMs, ferramentas e boas instruções! 🤓
> 
> 📈 Impacto? Adoção de IA pode impulsionar o PIB do Brasil e transformar a forma como trabalhamos, liberando tempo para tarefas mais criativas e estratégicas. 🤯
> 
> E aí, como você vê o futuro dos Agentes de IA na sua área? Compartilhe sua visão nos comentários! 👇
> 
> #InteligenciaArtificial #AgentesDeIA #Inovacao #Tecnologia


---------------------------------------------------------------

--- Resultado do Agente 4 (Revisor) ---



> O rascunho está ótimo, mas podemos deixá-lo ainda mais engajador para o público do Instagram com algumas pequenas alterações:
> 
> **Problemas e Sugestões:**
> 
> *   **Tom:** O tom está bom, mas podemos adicionar um pouco mais de entusiasmo e usar mais emojis para atrair a atenção.
> *   **Chamada para Ação:** Podemos deixar a chamada para ação mais direta e convidativa.
> 
> **Rascunho Revisado:**
> 
> 🚀 Prepare-se para a Revolução dos Agentes de IA! 🤖✨
> 
> Já ouviu falar dos Agentes de Inteligência Artificial? 🤔 Na Microsoft Build 2025, a mensagem foi clara: ELES SÃO O FUTURO! 🚀 Empresas como a B3 já estão a todo vapor com essa tecnologia, e o Brasil 🇧🇷 está investindo pesado em IA! 💰
> 
> ⚙️ O que eles fazem? Os Agentes de IA tomam decisões inteligentes e automatizam tarefas, aumentando a eficiência e a tomada de decisões em tempo real. Imagine no RH, finanças, vendas... As possibilidades são INACREDITÁVEIS! 🤩
> 
> 🛠️ Quer construir o seu próprio Agente? A OpenAI lançou um guia super prático e a IBM watsonx Orchestrate oferece soluções "no-code" para facilitar a sua vida! Componentes-chave: LLMs poderosos, ferramentas e instruções claras! 🤓
> 
> 📈 Impacto? A adoção de IA pode impulsionar o PIB do Brasil e transformar a forma como trabalhamos, liberando tempo para tarefas mais criativas e estratégicas. 🤯 Prepare-se para um futuro INOVADOR! ✨
> 
> E aí, como você IMAGINA o futuro dos Agentes de IA na sua área? Deixe sua opinião nos comentários! 👇
> 
> #InteligenciaArtificial #AgentesDeIA #Inovacao #Tecnologia #FuturoDaIA
> 


---------------------------------------------------------------
